![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


## Before you start

In order to complete the project you may wish to install some Hugging Face libraries such as `transformers` and `evaluate`.

In [5]:
# !pip install transformers
# !pip install evaluate==0.4.0
# !pip install datasets==2.10.0
# #!pip install sentencepiece==0.1.97
# !pip install sentencepiece

# from transformers import logging
# logging.set_verbosity(logging.WARNING)

In [72]:
#!pip install pandoc
#!pip install nbconvert

In [12]:
import pandas as pd
import torch, evaluate
from datasets import Dataset
from transformers import pipeline, AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering

## **LOAD DATA**

In [13]:
car_review = pd.read_csv("data/car_reviews.csv", sep=";")
car_review

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


In [14]:
# Convert Pandas DataFrame to Hugging Face Dataset
test_data = Dataset.from_pandas(car_review)
test_data

Dataset({
    features: ['Review', 'Class'],
    num_rows: 5
})

In [15]:
# from datacamp solution
# Put the car reviews and their associated sentiment labels in two lists
reviews_dc = car_review['Review'].tolist()
real_labels_dc = car_review['Class'].tolist()
real_labels_dc

['POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE']

## **SENTIMENT ANALYSIS MODEL**

### USING PIPELINE APPROACH

In [46]:
# My solution using pipeline

# Load the sentiment-analysis pipeline
classifier = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Run predictions
predicted_labels = classifier(test_data['Review'])

print(predicted_labels)

# Extract predicted labels
predictions = [1 if pred['label'] == 'POSITIVE' else 0 for pred in predicted_labels]

# Print predictions
print(predictions)

Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9293984174728394}, {'label': 'POSITIVE', 'score': 0.8654274344444275}, {'label': 'POSITIVE', 'score': 0.9994640946388245}, {'label': 'NEGATIVE', 'score': 0.9935314059257507}, {'label': 'POSITIVE', 'score': 0.9986565113067627}]
[1, 1, 1, 0, 1]


In [47]:
# datacamp solution using pipeline

# Load a sentiment analysis LLM into a pipeline
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Perform inference on the car reviews and display prediction results
predicted_labels_dc = classifier(reviews_dc)

# better way of presenting the prediction output
for review_dc, prediction_dc, label_dc in zip(reviews_dc, predicted_labels_dc, real_labels_dc):
    print(f"Review: {review_dc}\nActual Sentiment: {label_dc}\nPredicted Sentiment: {prediction_dc['label']} (Confidence: {prediction_dc['score']:.4f})\n")


Device set to use mps:0


Review: I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel tr

### USING AUTOMODEL APPROACH

In [48]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [49]:
# using tokenize function with map()

def tokenize_function(text_data):
    return tokenizer(text_data['Review'], 
                     return_tensors="pt", 
                     padding=True,
                     truncation=True, 
                     max_length=200)

# Tokenize in batches
tokenized_test_data = test_data.map(tokenize_function, batched=True)

print(type(tokenized_test_data))

print(tokenized_test_data)
#tokenized_test_data['input_ids'] # columns are not tensors, will need to convert to tensor

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['Review', 'Class', 'input_ids', 'attention_mask'],
    num_rows: 5
})


In [50]:
#using direct tokenizer call (output not used in the code below, just for illustrating the differences)

tokenized_test_data_with_direct_tokenizer_call = tokenizer(test_data['Review'], 
                     return_tensors="pt", 
                     padding=True,
                     #padding='max_length',
                     truncation=True, 
                     max_length=200)

print(type(tokenized_test_data_with_direct_tokenizer_call))

#tokenized_test_data_with_direct_tokenizer_call.input_ids # the columns are tensors

<class 'transformers.tokenization_utils_base.BatchEncoding'>


### Sentiment Analysis Prediction

In [51]:
# Assuming the model is already on the correct device (CPU or GPU)
device = model.device  # Get the device where the model is located

# Ensure tokenized_test_data['input_ids'] and tokenized_test_data['attention_mask'] are tensors
input_ids = torch.tensor(tokenized_test_data['input_ids']).to(device)
attention_mask = torch.tensor(tokenized_test_data['attention_mask']).to(device)

# Set model to evaluation mode
model.eval()

# Disable gradient computation to save memory
with torch.no_grad():
    predicted_labels_AM = model(input_ids=input_ids, attention_mask=attention_mask) #no need to use generate since not generating

# Check the model output structure
print(predicted_labels_AM)

# Extract logits from the model output
logits = predicted_labels_AM.logits  # This is usually a tensor of shape (batch_size, num_labels)

# Get the predicted labels by taking the argmax (class with the highest score)
predictions_AM = torch.argmax(logits, dim=1).tolist()

# Now, predicted_labels contains the predicted class indices for each example
print(predictions_AM)
# 	0: Negative
#	1: Positive

SequenceClassifierOutput(loss=None, logits=tensor([[-2.5634,  2.5600],
        [-0.8886,  0.9725],
        [-3.6580,  3.8730],
        [ 2.7306, -2.3037],
        [-1.5670,  1.6940]]), hidden_states=None, attentions=None)
[1, 1, 1, 0, 1]


### Sentiment Analysis Evaluation

In [52]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [53]:
# my solution

real_labels = [1 if data["Class"] == "POSITIVE" else 0 for data in test_data]
print(f"Real Labels: {real_labels}")
print(f"Predicted Labels: {predictions}")

accuracy_result_compute = accuracy.compute(references=real_labels, predictions=predictions)
accuracy_result = accuracy_result_compute['accuracy']
print(accuracy_result_compute)
f1_result_compute = f1.compute(references=real_labels, predictions=predictions)
f1_result = f1_result_compute['f1']
print(f1_result_compute)

Real Labels: [1, 0, 1, 0, 1]
Predicted Labels: [1, 1, 1, 0, 1]
{'accuracy': 0.8}
{'f1': 0.8571428571428571}


In [54]:
## datacamp solution

# Map categorical sentiment labels into integer labels
references_dc = [1 if label == "POSITIVE" else 0 for label in real_labels_dc]
predictions_dc = [1 if label['label'] == "POSITIVE" else 0 for label in predicted_labels_dc]

print(f"Real Labels: {references_dc}")
print(f"Predicted Labels: {predictions_dc}")

# Calculate accuracy and F1 score
accuracy_result_dict_dc = accuracy.compute(references=references_dc, predictions=predictions_dc)
accuracy_result_dc = accuracy_result_dict_dc['accuracy']
f1_result_dict_dc = f1.compute(references=references_dc, predictions=predictions_dc)
f1_result_dc = f1_result_dict_dc['f1']
print(f"Accuracy: {accuracy_result_dc}")
print(f"F1 result: {f1_result_dc}")

Real Labels: [1, 0, 1, 0, 1]
Predicted Labels: [1, 1, 1, 0, 1]
Accuracy: 0.8
F1 result: 0.8571428571428571


## **TEXT TRANSLATION MODEL**

### USING PIPELINE APPROACH

In [17]:
# my solution usine pipeline

# 	•	Here, truncation happens manually before passing the text to the model.
	# •	The model never sees the truncated part.
	# •	If important context is cut off, the translation might lose meaning or fluency.

# first 2 sentences of the 1st review
english_data = ['I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use.']

translator = pipeline(task="translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")

translated_review_out = translator(english_data, clean_up_tokenization_spaces=True)

translated_review = translated_review_out[0]['translation_text']
print(translated_review)
# for sentence in translated_review_out:
#     translated_review = sentence['translation_text']
#     print(translated_review)
# #print(output[0]["translation_text"])

Device set to use mps:0


Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.


In [27]:
# datacamp solution using pipeline

# Truncation Inside Translator
# Here, truncation happens inside the translator() function.
# 	•	The model processes the entire first_review and only outputs up to 27 tokens.
# 	•	The model determines which part of the text is most important for translation based on its learned attention mechanism.

# Best Practice: If possible, let the model handle truncation (max_length), unless you’re sure the input is too long for the model. 🚀


# Load translation LLM into a pipeline and translate car review
first_review = reviews_dc[0]
translator = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")

# max_length=27 to limit to only the first 2 sentences of the 1st review
translated_review_dc = translator(first_review, max_length=27)[0]['translation_text']
print(f"Model translation:\n{translated_review_dc}")

Device set to use mps:0
Your input_length: 365 is bigger than 0.9 * max_length: 27. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Model translation:
Estoy muy satisfecho con mi 2014 Nissan NV SL. Uso esta furgoneta para mis entregas de negocios y uso personal.


### USING AUTOMODEL APPROACH

In [55]:
translation_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")
translation_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")

english_input = translation_tokenizer(english_data, 
                     return_tensors="pt", 
                     padding=True,
                     truncation=True, 
                     max_length=100)

# Set model to evaluation mode
translation_model.eval()

# Disable gradient computation to save memory
with torch.no_grad():
    translated_review_out = translation_model.generate(**english_input) # using generate to generate translation

print(translated_review_out)
    
# Decode the translated tokens to get the translated text
translated_review_decoded = [translation_tokenizer.decode(tokens, skip_special_tokens=True) for tokens in translated_review_out]

#print(translated_review_decoded)

translated_review = translated_review_decoded[0]
print(translated_review)
# Print the translated sentences
# for sentence in translated_review_out:
#     translated_review = sentence
#     print(translated_review)

/Users/sumitsinha/miniconda3/envs/pytorch/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


tensor([[65000,  1686,   239, 22669,    29,   155, 40906,   716,  1239,   175,
           399,  8356,  9652,   135, 26327,    24,  1071,  3678,     9,     4,
          4553,    11,   694,   429,     3,     0]])
Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.


### Text Translation Evaluation

In [56]:
# Open and read the file into a list
with open("data/reference_translations.txt", "r") as file:
    reference_translations = file.readlines()

# Strip newlines and any extra spaces
reference_translations = [line.strip() for line in reference_translations] # each line is a list

# Print the list
print(f"Spanish translation references:\n{reference_translations}")

print(f"English Text:\n{translated_review}")

Spanish translation references:
['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.', 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']
English Text:
Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.


In [57]:
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(predictions=[translated_review], references=[reference_translations])
print(bleu_score)

{'bleu': 0.7794483794144497, 'precisions': [0.9090909090909091, 0.8571428571428571, 0.75, 0.631578947368421], 'brevity_penalty': 1.0, 'length_ratio': 1.0476190476190477, 'translation_length': 22, 'reference_length': 21}


In [25]:
# datacamp solution (different blue score from my solution because the datacamp solution translated text is slight different)

# Load and calculate BLEU score metric
bleu_score = bleu.compute(predictions=[translated_review_dc], references=[reference_translations])
print(bleu_score['bleu'])

0.6022774485691839


## **EXTRACTIVE Q&A MODEL**

In [56]:
test_data['Review'][1]

"The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found."

### USING PIPELINE APPROACH

In [21]:
# Using Pipeline - this is actually extractive Q&A Model

question = "What did he like about the brand?"
context = test_data['Review'][1]

# Define the appropriate model
qa = pipeline(task="question-answering", model='deepset/minilm-uncased-squad2')

output = qa(question=question, context=context)
print(output['answer'])

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


ride quality, reliability


### USING AUTOMODEL APPROACH

In [63]:
# My solution using AutoModel

# Load model and tokenizer
QAmodel = AutoModelForQuestionAnswering.from_pretrained('deepset/minilm-uncased-squad2')
QAtokenizer = AutoTokenizer.from_pretrained('deepset/minilm-uncased-squad2')

# Define question and context
question = "What did he like about the brand?"
context = test_data['Review'][1]

# Tokenize input
inputs = QAtokenizer(question, context, return_tensors="pt", truncation=True, padding=True)

# Perform inference
QAmodel.eval()
with torch.no_grad():
    outputs = QAmodel(**inputs)

print(outputs)

# The model returns two sets of scores:
# 	•	start_logits: Scores indicating the probability of each token being the start of the answer.
# 	•	end_logits: Scores indicating the probability of each token being the end of the answer.
# These scores are logits (raw scores before applying softmax).

# Extract answer span
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# •	torch.argmax(start_scores): Finds the index of the token most likely to be the start of the answer.
# •	torch.argmax(end_scores): Finds the index of the token most likely to be the end of the answer.
# •	+1: Since Python slicing is exclusive on the right, adding 1 ensures we include the last token.
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores) + 1  # +1 to include the last token

print(start_index, end_index)

# Decode answer
# •	inputs["input_ids"][0]: Retrieves the token IDs corresponding to the input text.
# •	convert_ids_to_tokens(...): Converts token IDs back into words (subword tokens).
# •	convert_tokens_to_string(...): Joins the tokens into a human-readable sentence.
#answer = QAtokenizer.convert_tokens_to_string(QAtokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index]))
#OR
answer = QAtokenizer.decode(inputs["input_ids"][0][start_index:end_index])

print("Answer:", answer)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 2.2762, -6.2343, -6.0321, -6.0678, -6.0882, -6.0398, -6.0916, -6.3243,
         -6.6169,  2.2762, -4.1784, -5.3074, -6.0259, -5.0759, -6.3563, -4.7436,
         -6.3526, -6.4298, -5.7884, -5.7856, -5.1914, -6.4162, -4.9813, -5.8880,
         -6.1990, -6.5654, -5.0210, -6.2009, -6.6289, -6.4944, -5.6861, -6.2675,
         -6.1552, -6.4367, -6.4137, -5.1342, -5.3643, -6.2586, -5.5127, -6.2846,
         -6.5086, -6.5867, -5.4555, -5.4227, -6.3117, -6.0545, -6.1566, -6.4791,
         -6.6384, -6.8284, -6.2884, -5.4548, -5.0252, -6.0445, -6.5075, -5.9411,
         -6.0409, -6.8019, -6.3508, -5.8416, -6.7474, -6.7375, -6.0882, -6.3069,
         -5.2681, -5.4771, -6.5445, -6.5645, -6.5778, -6.5040, -5.7578, -5.9364,
         -6.3930, -6.5072, -6.7121, -6.6489, -5.7286, -6.0741, -6.1914, -5.8883,
         -6.2878, -6.8106, -6.4051, -6.0894, -6.1538, -6.4427, -6.1129, -6.5051,
         -6.7101, -6.6582, -3.8438, -4.4204, -5.9236, -5

In [33]:
# datacamp solution using AutoModel - similar to my solution

# Instantiate model and tokenizer
model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

# Define context and question, and tokenize them
context = reviews_dc[1]
print(f"Context:\n{context}")
question = "What did he like about the brand?"
inputs = tokenizer(question, context, return_tensors="pt")

# Perform inference and extract answer from raw outputs
with torch.no_grad():
  outputs = model(**inputs)
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]

# Decode and show answer
answer = tokenizer.decode(answer_span)
print("Answer: ", answer)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Context:
The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.
Answer:  ride quality, reliability


## TEXT SUMMARIZATION MODEL

In [65]:
#last review
#test_data['Review'][4]
#OR
test_data['Review'][-1]  # much better way of finding last review

"I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of lane changes. Th

### USING PIPELINE APPROACH

In [66]:
# My solution using pipeline

summarizer = pipeline(task="summarization", model="facebook/bart-large-cnn")

summarized_text = summarizer(test_data['Review'][-1], max_length=51, clean_up_tokenization_spaces=True)
print(summarized_text[0]["summary_text"])

Device set to use mps:0
Your min_length=56 must be inferior than your max_length=51.
/Users/sumitsinha/miniconda3/envs/pytorch/lib/python3.11/site-packages/transformers/generation/utils.py:1432: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (51). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more.


In [40]:
# datacamp solution using pipeline (uses a different model)

# Get original text to summarize upon car review
text_to_summarize = reviews_dc[-1]
print(f"Original text:\n{text_to_summarize}")

# Load summarization pipeline and perform inference
model_name = "cnicu/t5-small-booksum" 
summarizer = pipeline("summarization", model=model_name)

outputs = summarizer(text_to_summarize, max_length=53)
summarized_text = outputs[0]['summary_text']
print(f"Summarized text:\n{summarized_text}")

Original text:
I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of la

Device set to use mps:0


Summarized text:
the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. I have hauled 12 bags of mulch in the back with the seats down and could have held more.


### USING AUTOMODEL APPROACH

In [67]:
# My solution using AutoModel

# Load model and tokenizer
model_name = "facebook/bart-large-cnn"
summarytokenizer = AutoTokenizer.from_pretrained(model_name)
summarymodel = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Select the text to summarize
input_text = test_data['Review'][-1]

# Tokenize input
inputs = summarytokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

# Generate summary
summarymodel.eval()

# The length_penalty parameter in generate() controls the preference for longer or shorter sequences in text generation.
# length_penalty=2.0 encourages shorter outputs because a higher penalty (>1.0) discourages longer sequences.
# •	If length_penalty < 1.0 → Favors longer outputs.
# •	If length_penalty > 1.0 → Favors shorter outputs.
# •	If length_penalty = 1.0 → No length penalty applied (default behavior).

with torch.no_grad():
    summary_ids = summarymodel.generate(**inputs, max_length=51, length_penalty=2.0)

# Decode summary
summarized_text = summarytokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("Summary:", summarized_text)

Summary: The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more.


## TEXT GENERATION MODEL

In [7]:
# Using Pipeline

from transformers import pipeline

generator = pipeline(task="text-generation", model="distilgpt2")
prompt = "The Gion neighborhood in Kyoto is famous for"
output = generator(prompt, max_length=100, pad_token_id=generator.tokenizer.eos_token_id)
print(" ".join(output[0]["generated_text"].split()))

Device set to use mps:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The Gion neighborhood in Kyoto is famous for its art and its colorful landscape. It's not the only place to have a family or a friend in Gion, Japan. In the city's northern suburbs, it is a unique and unique place to have a family or a friend anywhere. The name Gion refers to Gionshi Shiii in Japan. In the city's central city, it's the second-highest ranking family in the country. Other cities have the highest


In [29]:
# Using AutoModel - takes too much memory - times out without torch_dtype=torch.float16
# torch.float16: load the model in half-precision (fp16). This significantly reduces memory consumption.
# But not very good text generations

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# Define prompt
prompt = "The Gion neighborhood in Kyoto is famous for"
#input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# OR
input_ids = tokenizer.encode(prompt, return_tensors="pt")
print(input_ids)

device = torch.device("cpu")
model.to(device)
input_ids = input_ids.to(device)

model.eval()

# Generate text
with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
print(output_ids)   
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(output_text.strip())

tensor([[  464,   402,   295,  6232,   287, 36298,   318,  5863,   329]])
tensor([[  464,   402,   295,  6232,   287, 36298,   318,  5863,   329,   663,
          1029,    12, 17163,    11,  1029,    12, 17163,    11,   290,  1029,
            12, 17163,  6832,    13,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198]])
The Gion neighborhood in Kyoto is famous for its high-rise, high-rise, and high-rise buildings.


## GENERATIVE Q&A MODEL

In [30]:
# Using Pipeline - CORRECT extractive Q&A Model with using deepset/minilm-uncased-squad2

question = "What did he like about the brand?"
context = test_data['Review'][1]

# Define the appropriate model
qa = pipeline(task="question-answering", model='deepset/minilm-uncased-squad2')

output = qa(question=question, context=context)
print(output['answer'])

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


ride quality, reliability


In [32]:
# Using pipeline - CORRECT generative Q&A Model with using gpt2

question = "What did he like about the brand?"
context = test_data['Review'][1]

# Define the appropriate model
qa = pipeline(task="question-answering", model='gpt2')

input_text = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"

output = qa({"context": context, "question": question}, max_length=150)
answer = output['answer']

print(answer)

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0


 rest of issues found.


In [35]:
# Using AutoModel APproach for generative QA

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define the question and context
question = "What did he like about the brand?"
context = test_data['Review'][1]

# Load the model and tokenizer (using GPT-2 for generative Q&A)
model_name = "gpt2"  # Or use a larger model like "gpt-3.5-turbo" if available
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the pad_token to be the eos_token for GPT-2
tokenizer.pad_token = tokenizer.eos_token

# Prepare the input text combining question and context
input_text = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"

# Tokenize the input
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=1024)

# Generate the answer (sampling or beam search can be added for diversity)
# max_new_tokens ensures that the model generates no more than the specified number of tokens.
# num_return_sequences=1: This specifies the number of different sequences the model should generate for the given input. 
# In this case, it will generate 1 sequence (one possible answer).
# If you set it to num_return_sequences=3, the model will generate 3 distinct answers.
# no_repeat_ngram_size=2: you are asking the model not to repeat any bigrams (two consecutive tokens) during the generation process.

model.eval()
with torch.no_grad():
    outputs = model.generate(inputs['input_ids'], max_new_tokens=50, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode the generated answer
generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract only the answer part (after "Answer:")
answer_start = generated_answer.find("Answer:") + len("Answer:")
answer = generated_answer[answer_start:].strip()

print("Answer:", answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: I like the way the company is handling the situation. I think the problem is that the dealership is not doing enough to address the problems. They are not taking the time to fix the cars. There are a lot of problems with the vehicles, so
